### Transport Tutorial - Notebook 2

# Implement and solve a variation of *Dantzig's Transport Problem*

<img style="float: right; height: 80px;" src="_static/python.png">

## Aim and scope of this tutorial

This tutorial takes you through the steps to solve a simple optimization model
using the **ixmp4** database management package and the **linopy** optimization package.

We use **Dantzig's transport problem**, which is used as a [tutorial for linopy](https://linopy.readthedocs.io/en/latest/transport-tutorial.html).
This problem solves for a least-cost shipping schedule that meets demand constraints in several markets (cities)
and supply constraints at factories.

For reference of the transport problem, see:
> Dantzig, G B, Chapter 3.3. In Linear Programming and Extensions.  
> Princeton University Press, Princeton, New Jersey, 1963.

## Tutorial outline

This tutorial consists of three Jupyter notebooks:

0. Set up an **ixmp4.Platform** to store the scenario input data and solution
1. Implement the **baseline version of the transport problem** and solve it
2. Create an **alternative scenario** and solve it 

<div class="alert alert-info">

This notebook requires that you created and solved the base transport problem as shown in [**Notebook 1**](1_transport-tutorial.ipynb).

</div>

### Launching the platform and loading a `Run` from the **ixmp4** database instance

We launch a platform instance and display all models/scenarios currently stored in the connected database instance. We use the same local database we created in [Notebook 0](0_transport-tutorial_platform-setup.ipynb), so please see there for how to create such a database. 

In [ ]:
import ixmp4

platform = ixmp4.Platform("transport-tutorial")

To see which `Run` we can work on, we want to list all `Run`s available on this platform.

A platform can hold different versions of the same `Run`. One of these versions is usually assigned as the "default version". 

Whenever you load a `Run` from the platform, this "default version" is returned (unless you specifically ask for another).

Similarly, when you list all `Run` available from a platform, it shows only the "default versions" of the `Run`s per default.

Since this tutorial is only dealing with one `Run` and just one version of said `Run`, we have not declared it the "default version".

Thus, when listing or tabulating the available `Run`s, we need to specify that we also want to see non-default versions.

Here, we are using `tabulate()` to get an immediate tabular overview. Using `list()` instead would grant us immediate access to the `Run` objects.

In [ ]:
# Tabulate all Runs in the database
platform.runs.tabulate(default_only=False)

In [ ]:
# Model and scenario name we used for Dantzig's transport problem in part 1
model = "transport problem"
scenario = "standard"

If you have run the first part of tutorial before, the existing `Run` should have appeared, and we can load it.
Uncomment and run the following lines as appropriate:

In [ ]:
# Load the default version of the Run created in the first tutorial
# run = platform.runs.get(model=model, scenario=scenario)

# If you already solved this Run, remember to remove its solution!
# run.optimization.remove_solution()

If the `Run` did not appear (e.g. because you are starting with this tutorial), we can use a function that creates the `Run` from scratch in one step and solve it as usual:

In [ ]:
from tutorial.transport.utils import create_default_dantzig_run

run = create_default_dantzig_run(platform=platform)

We solve the `Run` as we did in [Notebook 1](1_transport-tutorial.ipynb).

In [ ]:
from tutorial.transport.dantzig_model_linopy import (
    create_dantzig_model,
    read_dantzig_solution,
)

linopy_model = create_dantzig_model(run=run)
linopy_model.solve("highs")

read_dantzig_solution(model=linopy_model, run=run)

### Retrieve some data from the `Run` for illustration of filters

Before cloning a `Run` and editing data, this section illustrates two-and-a-half methods to retrieve data for a `Parameter` from a `Run`.

In [ ]:
# Load the distance Parameter
d = run.optimization.parameters.get("d")
d

This is an `ixmp4.Parameter` object! It's useful to interact with if you want to edit the information stored in the database about it. You could e.g. add `.docs` to it so that you can always look up in the database what this object does.
For interacting with the modeling data, it's more useful to interact with the `.data` attribute, which stores the actual data: 

In [ ]:
d.data

This is a dictionary because that's easier to store in a database. For ease of access, you may want to convert it to a `pandas.DataFrame`:

In [ ]:
import pandas as pd

d_data = pd.DataFrame(d.data)

# Show only the distances for connections from Seattle by filtering the pandas.DataFrame
d_data[d_data["i"] == "seattle"]

In [ ]:
# NOTE We currently don't support loading only specific parameter elements.
# We always load the whole parameter and can then select from the data as usual with
# e.g. pandas.

For faster access or more complex filtering, you can then use all familiar `pandas` features. For example, list only distances from Seattle to specific other cities:

In [ ]:
d_data.loc[(d_data["i"] == "seattle") & (d_data["j"].isin(["chicago", "topeka"]))]

Please note that `pandas` recommends to use [advanced indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html#advanced-advanced-hierarchical) if you find yourself using more than three conditionals in `.loc[]`.

### Make a clone of the baseline scenario, then check out the clone and edit the scenario

For illustration of a scenario analysis workflow, we add a new demand location ``detroit`` and add a demand level and transport costs to that city.
Because the production capacity does not allow much slack for increased production, we also reduce the demand level in ``chicago``.

In [ ]:
# Create a new Run by cloning the existing one (without keeping the solution)
run_detroit = run.clone(model=model, scenario="detroit", keep_solution=False)

In [ ]:
# Reduce demand in chicago
run_detroit.optimization.parameters.get("b").add(
    {"j": ["chicago"], "values": [200], "units": ["cases"]}
)

# Add a new city with demand and distances
run_detroit.optimization.indexsets.get("j").add("detroit")
run_detroit.optimization.parameters.get("b").add(
    {"j": ["detroit"], "values": [150], "units": ["cases"]}
)
run_detroit.optimization.parameters.get("d").add(
    {
        "i": ["seattle", "san-diego"],
        "j": ["detroit", "detroit"],
        "values": [1.7, 1.9],
        "units": ["km", "km"],
    }
)

### Solve the new `Run`

In [ ]:
linopy_model_detroit = create_dantzig_model(run=run_detroit)
linopy_model_detroit.solve("highs")

read_dantzig_solution(model=linopy_model_detroit, run=run_detroit)

### Display and analyze the results

For comparison between the baseline `Run`, i.e., the original transport problem, and the "detroit" `Run`, we show the solution for both cases.

In [ ]:
# Display the objective value of the solution in the baseline Run
run.optimization.variables.get("z").levels

In [ ]:
# Display the objective value of the solution in the "detroit" Run
run_detroit.optimization.variables.get("z").levels

In [ ]:
# Display quantities transported from canning plants to demand locations in "baseline"
pd.DataFrame(run.optimization.variables.get("x").data)

In [ ]:
# Display quantities transported from canning plants to demand locations in "detroit"
pd.DataFrame(run_detroit.optimization.variables.get("x").data)

In [ ]:
# Display quantities and marginals (=shadow prices) of the demand balance constraints
# in "baseline"
run.optimization.equations.get("demand").data

In [ ]:
# Display quantities and marginals (=shadow prices) of the demand balance constraints
# in "detroit"
run_detroit.optimization.equations.get("demand").data